# Ejercicio 2 

## Generacion de Base de datos

In [1]:
# Importacion de modulo
%load_ext sql

In [2]:
# Base de datos
%sql sqlite:///parcial1

u'Connected: @parcial1'

In [3]:
# Muestra organizacion 
%sql pragma foreign_keys = on;

 * sqlite:///parcial1
Done.


[]

In [4]:
%%sql CREATE TABLE Estudiantes (
    idEstudiante VARCHAR(255) PRIMARY KEY, 
    nombre VARCHAR(255), 
    apellido VARCHAR(255), 
    carrera VARCHAR(255)
    );

 * sqlite:///parcial1
Done.


[]

In [5]:
%%sql CREATE TABLE Jerarquias (
    idFacultad VARCHAR(255) NOT NULL,
    idDepartamento VARCHAR(255) NOT NULL,
    nombreFacultad VARCHAR(255),
    nombreDepartamento VARCHAR(255),
    CONSTRAINT PK_Jerarquias PRIMARY KEY (idFacultad,idDepartamento)
    );


 * sqlite:///parcial1
Done.


[]

In [6]:
%%sql CREATE TABLE Cursos (
    idCurso VARCHAR(255) PRIMARY KEY, 
    nombre VARCHAR(255), 
    idDepartamento VARCHAR(255), 
    FOREIGN KEY (idDepartamento) REFERENCES Departamentos(idDepartamento)
    );

 * sqlite:///parcial1
Done.


[]

In [7]:
%%sql CREATE TABLE Profesores (
    idProfesor VARCHAR(255) PRIMARY KEY, 
    nombre VARCHAR(255), 
    apellido VARCHAR(255)
    );

 * sqlite:///parcial1
Done.


[]

In [8]:
%%sql CREATE TABLE Notas (
    idEstudiante VARCHAR(255) NOT NULL,
    idCurso VARCHAR(255) NOT NULL,
    fecha DATE,
    nota INT, 
    FOREIGN KEY (idEstudiante) REFERENCES Estudiantes(idEstudiante),
    FOREIGN KEY (idCurso) REFERENCES Cursos(idCurso)
    );

 * sqlite:///parcial1
Done.


[]

In [9]:
%%sql CREATE TABLE Directores (
    idProfesor VARCHAR(255) NOT NULL,
    idDepartamento VARCHAR(255) NOT NULL PRIMARY KEY,
    FOREIGN KEY (idDepartamento) REFERENCES Jerarquias(idDepartamento),
    FOREIGN KEY (idProfesor) REFERENCES Profesores(idProfesor)
    );

 * sqlite:///parcial1
Done.


[]

In [10]:
%%sql CREATE TABLE Asignacion (
    seccion VARCHAR(255) NOT NULL,
    idEstudiante VARCHAR(255) NOT NULL,
    idCurso VARCHAR(255) NOT NULL,
    idProfesor VARCHAR(255) NOT NULL,
    fecha DATE,
    FOREIGN KEY (idEstudiante) REFERENCES Estudiantes(idEstudiante),
    FOREIGN KEY (idCurso) REFERENCES Cursos(idCurso),
    FOREIGN KEY (idProfesor) REFERENCES Profesores(idProfesor)
    );

 * sqlite:///parcial1
Done.


[]

## Llenado con datos

In [11]:
# se meten datos en Estudiantes
%sql INSERT INTO Estudiantes (idEstudiante, nombre, apellido, carrera) VALUES ('1', 'Activo', 'Activo Apellido 1', 'Carrera activa 1 '); 
%sql INSERT INTO Estudiantes (idEstudiante, nombre, apellido, carrera) VALUES ('2', 'Activo 2', 'Activo Apellido 2', 'Carrera activa 2 '); 
%sql INSERT INTO Estudiantes (idEstudiante, nombre, apellido, carrera) VALUES ('3', 'Activo 3', 'Activo Apellido 3', 'Carrera activa 2 '); 
%sql INSERT INTO Estudiantes (idEstudiante, nombre, apellido, carrera) VALUES ('4', 'No Activo 1 ', 'No Activo Apellido 1', 'Carrera inactiva 1 '); 

 * sqlite:///parcial1
1 rows affected.
 * sqlite:///parcial1
1 rows affected.
 * sqlite:///parcial1
1 rows affected.
 * sqlite:///parcial1
1 rows affected.


[]

In [12]:
# se meten datos en Jerarquias
%sql INSERT INTO Jerarquias (idFacultad, idDepartamento, nombreFacultad, nombreDepartamento) VALUES ('J1', 'D1', 'NJ1', 'ND1'); 
%sql INSERT INTO Jerarquias (idFacultad, idDepartamento, nombreFacultad, nombreDepartamento) VALUES ('J1', 'D2', 'NJ1', 'ND2'); 
%sql INSERT INTO Jerarquias (idFacultad, idDepartamento, nombreFacultad, nombreDepartamento) VALUES ('J2', 'D1', 'NJ2', 'ND1'); 
%sql INSERT INTO Jerarquias (idFacultad, idDepartamento, nombreFacultad, nombreDepartamento) VALUES ('J2', 'D2', 'NJ2', 'ND2'); 

 * sqlite:///parcial1
1 rows affected.
 * sqlite:///parcial1
1 rows affected.
 * sqlite:///parcial1
1 rows affected.
 * sqlite:///parcial1
1 rows affected.


[]

In [13]:
# se meten datos en Cursos
%sql INSERT INTO Cursos (idCurso, nombre, idDepartamento) VALUES ('1', 'CURSO1', 'D1');
#%sql INSERT INTO Cursos (idCurso, nombre, idDepartamento) VALUES ('2', 'CURSO2', 'D2');
#%sql INSERT INTO Cursos (idCurso, nombre, idDepartamento) VALUES ('3', 'CURSO3', 'D1');

 * sqlite:///parcial1
(pysqlite2.dbapi2.OperationalError) no such table: main.Departamentos [SQL: u"INSERT INTO Cursos (idCurso, nombre, idDepartamento) VALUES ('1', 'CURSO1', 'D1');"] (Background on this error at: http://sqlalche.me/e/e3q8)


## Queries

### La cantidad de estudiantes actualmente activos por carrera

In [25]:
%%sql
SELECT carrera, count(*) as Cantidad_Estudiantes
FROM Estudiantes INNER JOIN Asignacion  ON Estudiantes.idEstudiante=Asignacion.idEstudiante
WHERE fecha BETWEEN '2019-01-01' AND '2019-12-31'
GROUP BY carrera

 * sqlite:///parcial1
Done.


carrera,Cantidad_Estudiantes


### Promedio de notas del estudiante Juan Pérez por cada semestre estudiado en la universidad

In [27]:
%%sql
SELECT Asignacion.fecha, AVG(Notas.nota) AS PROMEDIO
FROM  Notas INNER JOIN (Estudiantes INNER JOIN Asignacion ON Estudiantes.idEstudiante=Asignacion.idEstudiante) as SUB ON Notas.idEstudiante=SUB.idEstudiante
WHERE Estudiantes.nombre = 'Juan Pérez'
GROUP BY Asignacion.fecha

 * sqlite:///parcial1
Done.


fecha,PROMEDIO


### Promedio de notas del curso Física 2 por cada sección que fue abierta durante el primer semestre del 2017

In [35]:
%%sql 
SELECT Asignacion.seccion, AVG(nota) AS PROMEDIO
FROM  Cursos INNER JOIN (Notas INNER JOIN Asignacion ON Notas.idCurso=Asignacion.idCurso) as SUB ON Cursos.idCurso=SUB.idCurso
WHERE nombre = 'Física 2' and Asignacion.fecha BETWEEN '2017-01-01' AND '2017-06-30'
GROUP BY seccion

 * sqlite:///parcial1
Done.


seccion,PROMEDIO


### El listado de estudiantes candidatos para distinciones académicas del 2017 (es decir, aquellos cuyo promedio en ambos semestres del año fue mayor a 85)

In [75]:
%%sql 
SELECT nombre, apellido
FROM (Estudiantes INNER JOIN (
    SELECT Asignacion.idEstudiante
    FROM Notas INNER JOIN Asignacion ON Notas.idEstudiante=Asignacion.idEstudiante
    WHERE Asignacion.fecha BETWEEN '2017-01-01' AND '2017-06-30'
    GROUP BY Asignacion.idEstudiante 
    HAVING AVG(nota) > 85
    ) AS SUB ON Estudiantes.idEstudiante=SUB.idEstudiante)


 * sqlite:///parcial1
Done.


nombre,apellido


### El listado de estudiantes que van en limpio (es decir, que están actualmente activos y que no han perdido ningún curso).

In [90]:
%%sql
SELECT nombre, apellido
FROM Estudiantes INNER JOIN (
    SELECT Asignacion.idEstudiante
    FROM Asignacion INNER JOIN Notas ON Asignacion.idEstudiante=Notas.idEstudiante
    WHERE (Asignacion.fecha BETWEEN '2019-01-01' AND '2019-06-30') AND nota > 61
) AS SUB ON Estudiantes.idEstudiante=SUB.idEstudiante

 * sqlite:///parcial1
Done.


nombre,apellido
